<a href="https://colab.research.google.com/github/ovbystrova/dpl/blob/master/notebooks/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import Counter
import pickle


import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
df = pd.read_csv('/content/drive/My Drive/dpl_dataset.csv')
fakes = df[df['label']=='fake']
reals = df[df['label']=='real']

reals.reset_index(drop=True, inplace=True)

In [4]:
df['sampling'].value_counts()

No sampling    560000
nucleus        140000
top_k          140000
argmax         140000
temperature    140000
Name: sampling, dtype: int64

# Sentence Length 

In [5]:
# drop all that is less than 2 sentences
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

11466

In [6]:
len(ids_fdrop), len(ids_rdrop)

(11462, 5)

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [8]:
fakes.shape, reals.shape

((548534, 3), (548534, 3))

In [9]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3  BY A.J. Miller, Jr.  The three men will make t...  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [10]:
fakes.tail(), reals.tail()

(                                                     text label sampling
 548529  Jorg Badura for The New York Times The Lost Se...  fake   argmax
 548530  For a company with seven stores nationwide, Ja...  fake   argmax
 548531  Plenty has been written about the new iPhone 4...  fake   argmax
 548532  To prepare for the grandest day of them all, J...  fake   argmax
 548533  Teaching ideas based on New York Times content...  fake   argmax,
                                                      text label     sampling
 548529  Jorg Badura for The New York Times The Lost Se...  real  No sampling
 548530  For a company with seven stores nationwide, Ja...  real  No sampling
 548531  Plenty has been written about the new iPhone 4...  real  No sampling
 548532  To prepare for the grandest day of them all, J...  real  No sampling
 548533  Teaching ideas based on New York Times content...  real  No sampling)

In [0]:
# save only first two sentences

fakes['text'] = fakes['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))
reals['text'] = reals['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))

In [12]:
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

2047

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [14]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [15]:
fakes.tail(), reals.tail()

(                                                     text label sampling
 546482  Jorg Badura for The New York Times The Lost Se...  fake   argmax
 546483  For a company with seven stores nationwide, Ja...  fake   argmax
 546484  Plenty has been written about the new iPhone 4...  fake   argmax
 546485  To prepare for the grandest day of them all, J...  fake   argmax
 546486  Teaching ideas based on New York Times content...  fake   argmax,
                                                      text label     sampling
 546482  Jorg Badura for The New York Times The Lost Se...  real  No sampling
 546483  For a company with seven stores nationwide, Ja...  real  No sampling
 546484  Plenty has been written about the new iPhone 4...  real  No sampling
 546485  To prepare for the grandest day of them all, J...  real  No sampling
 546486  Teaching ideas based on New York Times content...  real  No sampling)

In [16]:
f = 0
for el in list(fakes['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        f +=1
r =0
for el in list(reals['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        r +=1
f, r

(0, 0)

# Drop duplicates

In [17]:
fakes.describe(), reals.describe()

(                                                     text   label sampling
 count                                              546487  546487   546487
 unique                                             543818       1        4
 top     This post has been corrected. The following is...    fake   argmax
 freq                                                   98  546487   138960,
                                                      text   label     sampling
 count                                              546487  546487       546487
 unique                                             519971       1            1
 top     This post is in partnership with Time. The art...    real  No sampling
 freq                                                  139  546487       546487)

In [18]:
fakes.shape, reals.shape

((546487, 3), (546487, 3))

In [0]:
fakes_clean = fakes.drop_duplicates(inplace=False)
reals_clean = reals.drop_duplicates(inplace=False)

In [20]:
fakes_clean.shape, fakes.shape, reals_clean.shape, reals.shape

((543961, 3), (546487, 3), (519971, 3), (546487, 3))

In [21]:
ids_fdrop = [ind for ind in fakes.index if ind not in fakes_clean.index]
ids_rdrop = [ind for ind in reals.index if ind not in reals_clean.index]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

28149

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [23]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [24]:
fakes.tail(), reals.tail()

(                                                     text label sampling
 518333  Americans are usually forgiving when they vote...  fake   argmax
 518334  Jorg Badura for The New York Times The Lost Se...  fake   argmax
 518335  For a company with seven stores nationwide, Ja...  fake   argmax
 518336  Plenty has been written about the new iPhone 4...  fake   argmax
 518337  To prepare for the grandest day of them all, J...  fake   argmax,
                                                      text label     sampling
 518333  Americans are usually forgiving when they vote...  real  No sampling
 518334  Jorg Badura for The New York Times The Lost Se...  real  No sampling
 518335  For a company with seven stores nationwide, Ja...  real  No sampling
 518336  Plenty has been written about the new iPhone 4...  real  No sampling
 518337  To prepare for the grandest day of them all, J...  real  No sampling)

In [25]:
fakes.describe()

,text,label,sampling
count,518338,518338,518338
unique,518330,1,4
top,Former President George H.W. Bush.,fake,nucleus
freq,3,518338,131708


In [26]:
reals.describe()

,text,label,sampling
count,518338,518338,518338
unique,518338,1,1
top,Dark Knight Rises villain Bane was a pretty fo...,real,No sampling
freq,1,518338,518338


# Counter

In [0]:
fakes_tokens = fakes['text'].apply(lambda x: x.split(' '))
reals_tokens = reals['text'].apply(lambda x: x.split(' '))

fake_split = Counter(x for xs in fakes_tokens for x in set(xs))
real_split = Counter(x for xs in reals_tokens for x in set(xs))

In [28]:
fake_split.most_common(20), real_split.most_common(20)

([('the', 424152),
  ('a', 355430),
  ('of', 329310),
  ('to', 320732),
  ('in', 299403),
  ('and', 286406),
  ('The', 200671),
  ('on', 185277),
  ('for', 180801),
  ('is', 172110),
  ('that', 167772),
  ('with', 130242),
  ('at', 125887),
  ('was', 125569),
  ('has', 109640),
  ('by', 97927),
  ('as', 95966),
  ('from', 95865),
  ('an', 91495),
  ('it', 79383)],
 [('the', 441593),
  ('a', 359409),
  ('of', 346475),
  ('to', 328792),
  ('and', 312185),
  ('in', 311877),
  ('on', 196144),
  ('for', 182471),
  ('that', 160730),
  ('The', 157442),
  ('is', 157035),
  ('with', 133134),
  ('at', 124429),
  ('was', 112411),
  ('has', 102500),
  ('from', 100164),
  ('as', 97920),
  ('by', 94024),
  ('an', 92637),
  ('his', 87801)])

# Save samplings datasets

In [29]:
samplings = fakes['sampling'].unique()

for sampling in samplings:
    
    fake_sampl = fakes[fakes['sampling']==sampling]
    real_sampl = reals.iloc[fake_sampl.index]
    assert real_sampl.shape == fake_sampl.shape, 'Different lengths of real and fake samplings'

    df_sampl = pd.concat([fake_sampl, real_sampl], axis=0)
    df_sampl = shuffle(df_sampl)
    df_sampl.reset_index(drop=True, inplace=True)
    
    df_sampl.to_csv('/content/drive/My Drive/data/sampling_{}.csv'.format(sampling), index=False)
    print('{} texts for {} sampling type were saved in sampling_{}.csv'.format(df_sampl.shape[0], sampling, sampling))

254220 texts for temperature sampling type were saved in sampling_temperature.csv
261200 texts for top_k sampling type were saved in sampling_top_k.csv
263416 texts for nucleus sampling type were saved in sampling_nucleus.csv
257840 texts for argmax sampling type were saved in sampling_argmax.csv


# Shuffle and save

In [0]:
df = pd.concat([fakes, reals], axis=0)
df.reset_index(drop=True, inplace=True)

In [31]:
df.head()

,text,label,sampling
0,"HAMBURG, Germany, June 3  As he left the socc...",fake,temperature
1,"WASHINGTON, Dec. 23 - The National Security Ag...",fake,temperature
2,IF outsized executive pay has indeed become a ...,fake,temperature
3,"BY A.J. Miller, Jr.",fake,temperature
4,Spinach has terrorized generations of veggie-p...,fake,temperature


In [32]:
df.tail()

,text,label,sampling
1036671,Americans are usually forgiving when they vote...,real,No sampling
1036672,Jorg Badura for The New York Times The Lost Se...,real,No sampling
1036673,"For a company with seven stores nationwide, Ja...",real,No sampling
1036674,Plenty has been written about the new iPhone 4...,real,No sampling
1036675,"To prepare for the grandest day of them all, J...",real,No sampling


In [0]:
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [34]:
df.head()

,text,label,sampling
0,"Jan. 15, 2016 6:56 p.m. ET Equities took anoth...",real,No sampling
1,Times Insider shares insights into how we work...,fake,top_k
2,is an unfit dog owner and her pit bull should ...,fake,temperature
3,LOS ANGELES – U.S. authorities on Tuesday depo...,fake,argmax
4,Just inside the front entrance of Southwest Ai...,real,No sampling


In [35]:
df.tail()

,text,label,sampling
1036671,NOT any of these guys! Megan Fox is giving us ...,real,No sampling
1036672,"From dressing up to carving pumpkins, celebrit...",real,No sampling
1036673,By MICHAEL FALCONE (@michaelpfalcone) and AMY ...,real,No sampling
1036674,Turkey's deputy prime minister told CNBC the c...,fake,top_k
1036675,"Answer by Auren Hoffman on Quora.com ""I'm a b...",fake,argmax


In [0]:
fake = list(fakes['text'])
real = list(reals['text'])

df.to_csv('/content/drive/My Drive/data/dataset.csv', index=False)

In [0]:
with open('/content/drive/My Drive/data/fake', 'wb') as fp:
    pickle.dump(fake, fp)

with open('/content/drive/My Drive/data/real', 'wb') as fp:
    pickle.dump(real, fp)

In [0]:
# with open ('fake', 'rb') as fp:
#     itemlist = pickle.load(fp)